In [ ]:
import os
from typing import cast, Dict, List, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.model_zoo as model_zoo
from torch import Tensor

# architecture

In [ ]:
__all__ = [
    "VGG",
    "vgg11", "vgg13", "vgg16", "vgg19",
    "vgg11_bn", "vgg13_bn", "vgg16_bn", "vgg19_bn",
]

vgg_cfgs: Dict[str, List[Union[str, int]]] = {
    "vgg11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "vgg13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "vgg16": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
    "vgg19": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"],
}

model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
}


def _make_layers(vgg_cfg: List[Union[str, int]], batch_norm: bool = False) -> nn.Sequential:
    layers: nn.Sequential[nn.Module] = nn.Sequential()
    in_channels = 3
    for v in vgg_cfg:
        if v == "M":
            layers.append(nn.MaxPool2d((2, 2), (2, 2)))
        else:
            v = cast(int, v)
            conv2d = nn.Conv2d(in_channels, v, (3, 3), (1, 1), (1, 1))
            if batch_norm:
                layers.append(conv2d)
                layers.append(nn.BatchNorm2d(v))
                layers.append(nn.ReLU(True))
            else:
                layers.append(conv2d)
                layers.append(nn.ReLU(True))
            in_channels = v

    return layers


class VGG(nn.Module):
    def __init__(self, vgg_cfg: List[Union[str, int]], batch_norm: bool = False, num_classes: int = 1000) -> None:
        super(VGG, self).__init__()
        self.features = _make_layers(vgg_cfg, batch_norm)

        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes),
        )

        # Initialize neural network weights
        self._initialize_weights()

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

    # Support torch.script function
    def _forward_impl(self, x: Tensor) -> Tensor:
        out = self.features(x)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.classifier(out)

        return out

    def _initialize_weights(self) -> None:
        for module in self.modules():
            if isinstance(module, nn.Conv2d):
                nn.init.kaiming_normal_(module.weight, mode="fan_out", nonlinearity="relu")
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.BatchNorm2d):
                nn.init.constant_(module.weight, 1)
                nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.Linear):
                nn.init.normal_(module.weight, 0, 0.01)
                nn.init.constant_(module.bias, 0)


def vgg11(pretrained=True, **kwargs) -> VGG:
    if pretrained:
        kwargs["init_weights"] = False
    model = VGG(vgg_cfgs["vgg11"], False, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls["vgg11"]))

    return model


def vgg13(pretrained=True, **kwargs) -> VGG:
    if pretrained:
        kwargs["init_weights"] = False
    model = VGG(vgg_cfgs["vgg13"], False, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls["vgg13"]))

    return model


def vgg16(pretrained=True, **kwargs) -> VGG:
    if pretrained:
        kwargs["init_weights"] = False
    model = VGG(vgg_cfgs["vgg16"], False, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls["vgg16"]))

    return model


def vgg19(pretrained=True, **kwargs) -> VGG:
    if pretrained:
        kwargs["init_weights"] = False
    model = VGG(vgg_cfgs["vgg19"], False, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls["vgg19"]))

    return model


def vgg11_bn(pretrained=True, **kwargs) -> VGG:
    if pretrained:
        kwargs["init_weights"] = False
    model = VGG(vgg_cfgs["vgg11"], True, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls["vgg11_bn"]))

    return model


def vgg13_bn(pretrained=True, **kwargs) -> VGG:
    if pretrained:
        kwargs["init_weights"] = False
    model = VGG(vgg_cfgs["vgg13"], True, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls["vgg13_bn"]))

    return model


def vgg16_bn(pretrained=True, **kwargs) -> VGG:
    if pretrained:
        kwargs["init_weights"] = False
    model = VGG(vgg_cfgs["vgg16"], True, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls["vgg16_bn"]))

    return model


def vgg19_bn(pretrained=True, **kwargs) -> VGG:
    if pretrained:
        kwargs["init_weights"] = False
    model = VGG(vgg_cfgs["vgg19"], True, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls["vgg19_bn"]))

    return model

# dataset

In [ ]:
import torchvision
import torchvision.transforms as transforms

'''
ImageNet
'''
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

'''
CIFAR-10 dataset
'''
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

# train / test def

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = vgg16().to(device)
model = torch.nn.DataParallel(model)
cudnn.benchmark = True

learning_rate = 0.1
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

file_name = 'vgg16_cifar10.pth'

def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current train average loss:', loss.item())

    print('\nTotal train accuracy:', 100. * correct / total)
    print('Total train loss:', train_loss)

def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    model.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))

    print('\nTotal test accuracy:', 100. * correct / total)
    print('Total test loss:', loss)

def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# train / test

In [ ]:
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)

# **vgg**
make class

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

**Data Loading**

cifar-100

In [ ]:
def data_loader(data_dir,
                batch_size,
                random_seed=42,
                valid_size=0.1,
                shuffle=True,
                test=False):
  
    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465], # R,G,B 평균
        std=[0.2023, 0.1994, 0.2010],  # R,G,B 표준편차
    )
    # normalize 후 R채널 결과 = (픽셀 값 - R채널 평균) / (R채널 표준편차)

    # define transforms, -1~1사이 값으로 정규화
    transform = transforms.Compose([
            transforms.Resize((227,227)), # 227X227로 변환
            transforms.ToTensor(),  # 데이터를 pytorch tensor로 변환
            normalize, 
    ])

    if test:
        dataset = datasets.CIFAR10(
          root=data_dir, train=False,
          download=True, transform=transform,
        )

        data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=shuffle
        )

        return data_loader

    # load the dataset
    '''
    train_dataset = datasets.CIFAR100(
        root=data_dir, train=True,
        download=True, transform=transform,
    )
   '''
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=transform,
    )


    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx) # 전체데이터셋 중 일부 샘플만 로딩
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler)
 
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler)

    return (train_loader, valid_loader)


# CIFAR100 dataset 
train_loader, valid_loader = data_loader(data_dir='./data',
                                         batch_size=64)

test_loader = data_loader(data_dir='./data',
                              batch_size=64,
                              test=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


**vgg16 generation**

In [ ]:
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()  # nn.Module 클래스 상속
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(64),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),  
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),  
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),  
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x): # vgg16
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

**HyperParameters**

In [ ]:
num_classes = 100 #cifar-100 10
num_epochs = 2 #20
batch_size = 16
learning_rate = 0.005

model = VGG16(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
# weight_decay: 가중치 regularization 방법, lambda에 해당되는 값
# momentum: gradient descent시 3차이상 곡선일 경우, 저점이 1개 이상일 떄 ,momentum을 이용 최저점으로 감
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  


# Train the model
total_step = len(train_loader)

In [ ]:
!nvidia-smi

Mon Jan 30 11:55:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    28W /  70W |  13388MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Training**


In [ ]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels) # CEE

        ##writer.add_scalar("Loss/train", loss, epoch)
        
        # Backward and optimize
        optimizer.zero_grad() # gradient를  계속 더하는 것을 방지(이전epoch)
        loss.backward()
        optimizer.step() # 경사하강법

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad(): # gradient 계산하지 않음
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total)) 

# 5 epoch --> 1시간소요(T4)  cifar100
'''

''' 
#writer.flush()

Epoch [1/2], Step [704/704], Loss: 0.9978
Accuracy of the network on the 5000 validation images: 56.38 %
Epoch [2/2], Step [704/704], Loss: 0.8654
Accuracy of the network on the 5000 validation images: 68.46 %


'\n\n'

In [ ]:
torch.save(model, f'./model.pt') # 모델저장

**Testing**

In [ ]:

with torch.no_grad(): # without backward
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images) # inference
        _, predicted = torch.max(outputs.data, 1) 
        # top5-accuracy
        top5_prob, top5_catid = torch.topk(outputs, 5) # 정확도가 높은 5개 category

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    # 테스트 41.57% (epoch 5)
    print(correct, total)
    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))   

0 10000
Accuracy of the network on the 10000 test images: 0.0 %


In [ ]:
top5_prob

tensor([[11.9592, 11.2877,  8.4408,  7.9247,  7.7505],
        [10.7870, 10.2661,  9.0566,  8.5584,  7.9923],
        [ 9.1069,  8.7674,  8.2718,  7.1638,  6.1711],
        [19.2843, 18.2723, 12.2174, 10.7249,  8.4604],
        [13.2295, 11.3599,  8.6078,  8.1303,  7.7163],
        [15.7751, 10.8772,  8.5164,  7.6755,  7.6438],
        [ 8.7259,  7.3943,  7.2434,  6.3402,  5.5717],
        [10.4497,  7.4743,  7.3699,  6.7751,  6.4071],
        [16.7717, 11.5816,  9.0891,  8.8626,  8.2402],
        [19.8406, 12.5433, 11.0494, 10.2840, 10.0168],
        [11.7720, 11.2332,  9.7175,  8.3426,  5.8863],
        [19.1812, 15.6708, 10.1156,  9.3436,  8.1404],
        [10.7642,  9.8972,  9.5395,  9.0037,  8.2372],
        [10.4976,  9.4794,  8.4420,  7.8343,  6.6443],
        [ 7.0895,  5.8935,  5.3626,  4.8164,  4.6634],
        [13.1631,  9.6788,  8.9621,  8.6100,  8.6000]], device='cuda:0')

In [ ]:
top5_catid

tensor([[3, 5, 2, 7, 6],
        [4, 7, 5, 2, 3],
        [6, 2, 4, 3, 0],
        [1, 9, 0, 8, 7],
        [7, 5, 3, 2, 4],
        [7, 4, 5, 2, 0],
        [4, 7, 2, 5, 3],
        [6, 3, 4, 9, 5],
        [7, 4, 0, 9, 5],
        [7, 4, 0, 5, 2],
        [1, 8, 9, 0, 4],
        [1, 9, 7, 8, 0],
        [5, 4, 3, 2, 7],
        [0, 2, 4, 7, 8],
        [9, 1, 7, 3, 0],
        [7, 4, 3, 2, 5]], device='cuda:0')

In [ ]:
torch.cuda.is_available() # GPU여부 체크

True

torch에 있는 vgg모델 로딩

In [ ]:
#  Very Deep Convolutional Networks for Large-Scale Image Recognition
# https://pytorch.org/hub/pytorch_vision_vgg/ 예제코드
import torch
#model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=True)
# or any of these variants
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11_bn', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg13', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg13_bn', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16_bn', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19_bn', pretrained=True)
model.eval()

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg11_bn-6002323d.pth" to /root/.cache/torch/hub/checkpoints/vgg11_bn-6002323d.pth


  0%|          | 0.00/507M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [ ]:
# Download an example image from the pytorch website
# 테스트이미지 로딩
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")

try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [ ]:
#image net 2012 dataset

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224), # 가운데를 중심으로 224 외의 범위 crop
    transforms.ToTensor(), # 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # R,G,B 평균,표준편차로 정규화
])
input_tensor = preprocess(input_image) # 전처리
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad(): # gradient 수정하지 않음(backward를 안함)
    output = model(input_batch)

# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
#print(output[0])
print(type(output[0]), len(output[0]))
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
#print(probabilities)
print(max(probabilities), torch.argmax(probabilities))

<class 'torch.Tensor'> 1000
tensor(0.3915, device='cuda:0') tensor(258, device='cuda:0')


In [ ]:
output.size()

torch.Size([1, 1000])

ImageNet classes


In [ ]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2023-01-30 10:39:02--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2023-01-30 10:39:02 (81.4 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



In [ ]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
# top5-accuracy
top5_prob, top5_catid = torch.topk(probabilities, 5) # 정확도가 높은 5개 category

# top-1 accuracy
top1_prob, top1_catid = torch.topk(probabilities, 1) # 정확도가 높은 1개 category

for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Samoyed 0.3915393650531769
Pomeranian 0.38202133774757385
wallaby 0.027552057057619095
schipperke 0.02753569185733795
Eskimo dog 0.02743017114698887


In [ ]:
top1_prob

_bn(batch norm version)
      top-1 error top-5 error
    
vgg11	    30.98	   11.3

vgg11_bn	26.70	   8.58

vgg13	    30.07	   10.75

vgg13_bn	28.45	   9.63

vgg16	    28.41	   9.62

vgg16_bn	26.63	   8.50

vgg19	    27.62	   9.12

vgg19_bn	25.76	   8.15